# Using the run `Manager` class

This notebook shows you how to use the `Manager` class to define a world and emit photons at various points within the crystal to test how the collection efficiency changes depending on where the photons are scintillated.

In [ ]:
import numpy as np
import pvtrace
import scintillator_tracer as st

# Verification

This allows you to examine the geometry and make sure everything looks good.

In [ ]:
nodes = st.worlds.build_rod_world()
world = nodes['world']
crystal = nodes['crystal']

crystal_mid = (st.geometry_helpers.max_physical_value(crystal, 'z')+st.geometry_helpers.min_physical_value(crystal, 'z'))/2
light = st.definitions.generate_scintillated_light_node(world, 'light')
light.location = (0, 0, crystal_mid)

scene = pvtrace.Scene(world)
all_ray_steps = st.processing.process_photons_with_visual(scene, num_photons=100, seed=10, open_browser=True)

## Event classification

This counts the fates of each photon, i.e. if and where they were absorbed.

In [ ]:
world = nodes['world']
world_nodes = st.processing.get_nodes_from_world(world).values()
print(world_nodes)
out = st.processing.organize_rays(all_ray_steps, world_nodes)
for key in out.keys():
    print(key, len(out[key]))
    print()
print(len(out.all))
print(out['absorb'][0][-1][0].position)
ray_pos = out['absorb'][0][-1][0].position

# Grid trials

This is where we release photons at various points within the crystal and see where we get the best and worst performance.

In [ ]:
nodes = st.worlds.build_rod_world()
crystal = nodes['crystal']

zrange = (
    st.geometry_helpers.min_physical_value(crystal, 'z')+0.5,
    st.geometry_helpers.max_physical_value(crystal, 'z')-0.5,
)
yrange = (
    st.geometry_helpers.min_physical_value(crystal, 'y')+0.1,
    st.geometry_helpers.max_physical_value(crystal, 'y')-0.1,
)

crystal_width = st.geometry_helpers.max_physical_value(crystal, 'y') - st.geometry_helpers.min_physical_value(crystal, 'y')
crystal_height = st.geometry_helpers.max_physical_value(crystal, 'z') - st.geometry_helpers.min_physical_value(crystal, 'z')
cut_depth = crystal_width / np.tan(np.radians(90-30))

This is when things get a little weird.
We need to define the `horizontal_func` keyword in the `define_grid` method.
This keyword specifies how the horizontal spacing (in this case, the `y` coordinate) of the grid points changes as a function of the vertical coordinate (in this case, `z`).

In [ ]:
steps = 5 # Generates a 5x5 grid
man = st.processing.Manager(
    st.worlds.build_rod_world,
    out_dir=f'rod/'
)
man.define_grid(
    vertical_range=zrange,
    horizontal_func=lambda z: yrange,
    steps=steps,
    visualize=True
)

## Run

This will simulate the scintillation release at each grid with `photons_per_grid_point` at each point.

In [ ]:
runs = man.run(photons_per_grid_point=200, seed=100, visualize=False)

Then, make the plots of what fraction of photons released at each grid were absorbed by each object.

In [ ]:
man.plot_all_ratios_absorbed()